In [ ]:
# Load required libraries
options(warn = -1)
suppressPackageStartupMessages({
  library(shiny)
  library(ggplot2)
  library(dplyr)
  library(plotly)
})

# Set seed for reproducibility
set.seed(123)

# Generate initial dataset
days <- 30
dates <- seq.Date(from = as.Date("2023-01-01"), by = "day", length.out = days)
Visits <- round(runif(days, min = 200, max = 700))
Sales <- round(Visits * runif(days, min = 0.1, max = 0.3))
Growth <- c(100, cumsum(diff(Sales)))

# Create a reactive dataset
sample_data <- reactiveVal(data.frame(
  Date = dates,
  Visits = Visits,
  Sales = Sales,
  Growth = Growth,
  stringsAsFactors = FALSE
))

# UI definition
ui <- fluidPage(
  titlePanel("📊 Shiny App: Real-Time Data Simulation"),
  
  sidebarLayout(
    sidebarPanel(
      sliderInput("days", "Select range of days:", 
                  min = 1, max = days, value = c(1, days), step = 1, animate = TRUE),
      selectInput("metric", "Analysis Variable:", choices = c("Visits", "Sales", "Growth")),
      checkboxInput("showTrend", "Show Trend Line", value = TRUE),
      actionButton("start", "Start Simulation"),
      actionButton("stop", "Stop Simulation"),
      sliderInput("interval", "Update Interval (seconds):", min = 1, max = 5, value = 2)
    ),
    
    mainPanel(
      tabsetPanel(
        tabPanel("📈 Sales Analysis", plotlyOutput("barPlotDynamic")),
        tabPanel("📊 Visits vs Sales", plotlyOutput("scatterPlotDynamic")),
        tabPanel("📄 Data Overview", tableOutput("dataTable"))
      )
    )
  )
)

# Server logic
server <- function(input, output, session) {
  
  # Reactive values for controlling simulation
  running <- reactiveVal(FALSE)
  
  # Function to update data dynamically
  observe({
    invalidateLater(input$interval * 1000, session) # Update based on interval
    if (running()) {
      new_date <- max(sample_data()$Date) + 1
      new_visits <- round(runif(1, min = 200, max = 700))
      new_sales <- round(new_visits * runif(1, min = 0.1, max = 0.3))
      new_growth <- tail(sample_data()$Growth, 1) + (new_sales - tail(sample_data()$Sales, 1))
      
      updated_data <- rbind(sample_data(), data.frame(
        Date = new_date, Visits = new_visits, Sales = new_sales, Growth = new_growth
      ))
      
      sample_data(updated_data)
    }
  })
  
  # Start and stop simulation
  observeEvent(input$start, { running(TRUE) })
  observeEvent(input$stop, { running(FALSE) })
  
  # Filter dataset based on selected range
  dataSubset <- reactive({
    start_index <- input$days[1]
    end_index <- input$days[2]
    sample_data()[start_index:end_index, ]
  })
  
  # Render bar plot
  output$barPlotDynamic <- renderPlotly({
    p <- ggplot(dataSubset(), aes(x = Date, y = .data[[input$metric]])) +
      geom_bar(stat = "identity", fill = "steelblue") +
      labs(title = paste("Daily", input$metric, "Over Time"), x = "Date", y = input$metric) +
      theme_minimal() +
      scale_x_date(date_labels = "%b %d", date_breaks = "1 week") +
      theme(axis.text.x = element_text(angle = 45, hjust = 1))
    
    ggplotly(p)
  })
  
  # Render scatter plot for Visits vs Sales
  output$scatterPlotDynamic <- renderPlotly({
    p <- ggplot(dataSubset(), aes(x = Visits, y = Sales)) +
      geom_point(color = "red", size = 3) +
      labs(title = "Sales vs Visits", x = "Number of Visits", y = "Number of Sales") +
      theme_minimal()
    
    if (input$showTrend) {
      p <- p + geom_smooth(method = "lm", color = "blue", se = FALSE, formula = y ~ x)
    }
    
    ggplotly(p)
  })
  
  # Render table for data overview
  output$dataTable <- renderTable({
    data <- dataSubset()
    data$Date <- format(data$Date, "%Y-%m-%d")  # Convertir la fecha a string legible
    data
  })
}

# Run the Shiny app
shinyApp(ui = ui, server = server)



Listening on http://127.0.0.1:7269

